# ETL_EDA
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from BPA 
The [dataset](https://transmission.bpa.gov/business/operations/Wind/baltwg.txt) is a continuously updated txt file in csv format. It contains the energy producation and load in the last 5 days. There will be some blank lines at the end for today. It can be retrieved simply by making `requests` without parameters. For more complicated API call, you may need to add query parameters.

In [9]:
#import requests

#url = "https://transmission.bpa.gov/business/operations/Wind/baltwg.txt"
#req = requests.get(url, timeout=0.5)
# req.raise_for_status()
# text = req.text
# print(text[:1000])
# print('...')
# print(text[-500:])

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from mongodb import *
import json

In [4]:
dbname = 'covid'
collection_name = 'cases'

date_str_to_num = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
                   'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
                   'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [5]:
def get_cases_data():
    url = "https://covidtracking.com/data/national/cases"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    data = []
    for tr in soup.body.div.div.main.find_all('tr')[1:]:
        raw = tr.find_all('span')
        date_str, case, new_case = raw[1].contents[0], raw[3].contents[0], raw[5].contents[0]
        month_str, day, year = date_str.replace(",", "").split(" ")
        month = date_str_to_num[month_str]
        date = "{}-{}-{}".format(year, month, day)
        data.append([date, case, new_case])
    df = pd.DataFrame(data=data, columns=['Date','Case','new_case'])
    return df


def create_collection(handle):
    collection = handle[collection_name]
    collection.drop()
    print('create collection {}'.format(collection_name))
    return collection


def insert(data, handle):
    records = json.loads(data.T.to_json()).values()
    handle.insert(records)

In [13]:
data = get_cases_data()
db = connect(dbname)
col = create_collection(db)
insert(data, col)

ConfigurationError: The "dnspython" module must be installed to use mongodb+srv:// URIs

In [12]:
!pip install dnspython

You should consider upgrading via the '/home/gitpod/.pyenv/versions/3.8.6/bin/python3 -m pip install --upgrade pip' command.


In [9]:
/home/gitpod/.pyenv/versions/3.8.6/bin/python3 -m pip install --upgrade pip

SyntaxError: invalid syntax (<ipython-input-9-edc796d548f0>, line 1)

In [5]:
# project
import requests
import pandas as pd
from bs4 import BeautifulSoup
#from mongodb import *
#import json
dbname = 'covid'
collection_name = 'cases'

In [6]:
# project
url = "https://covidtracking.com/data/national/cases"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
data = []
for tr in soup.body.div.div.main.find_all('tr')[1:]:
    raw = tr.find_all('span')
    data.append([raw[1].contents[0], raw[3].contents[0], raw[5].contents[0]])
df = pd.DataFrame(data=data, columns=['Date','Case','new_case'])

## 2. Raw Data to Documents/Dicts
Using pandas, it is simple to parse a in-memory string. The first few lines of description need to be skipped. Datetime conversion is made and blank lines are dropped. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [3]:
# import pandas
# from io import StringIO

# df = pandas.read_csv(StringIO(text), skiprows=11, delimiter='\t')
# df.columns = df.columns.str.strip()             # remove space in columns name
# df['Datetime'] = pandas.to_datetime(df['Date/Time'])
# df.drop(columns=['Date/Time'], axis=1, inplace=True)
# df.dropna(inplace=True)  
# df.head()

In [7]:
# project
import pandas as pd
df.columns = df.columns.str.strip()             # remove space in columns name
#df['Date'] = pd.to_datetime(str(df['Date']))
#df.drop(columns=['Date/Time'], axis=1, inplace=True)
df.dropna(inplace=True)  
df.head()

,Date,Case,new_case
0,"Nov 23, 2020","12,230,472","150,975"
1,"Nov 22, 2020","12,079,497","150,836"
2,"Nov 21, 2020","11,928,661","179,714"
3,"Nov 20, 2020","11,748,947","192,805"
4,"Nov 19, 2020","11,556,142","182,832"


## 3. Upsert MongoDB
If we fetch the data frequently, there are lots of duplicate data entry between each run. The de-duplication happens at insertion. The MongoDB API to use is `collection.replace_one(filter=..., replacement=..., upsert=True)`. The statement matches a document in MongoDB with `filter`, replaces it with `replacement` if the document exists or inserts `replacement` into the database if `filter` matches nothing. 

In [8]:
# import pymongo

# client = pymongo.MongoClient()

In [9]:
# db = client.get_database("energy")
# collection = db.get_collection("energy")
# update_count = 0
# for record in df.to_dict('records'):
#     result = collection.replace_one(
#         filter={'Datetime': record['Datetime']},    # locate the document if exists
#         replacement=record,                         # latest document
#         upsert=True)                                # update if exists, insert if not
#     if result.matched_count > 0:
#         update_count += 1
# print(f"rows={df.shape[0]}, update={update_count}, "
#       f"insert={df.shape[0]-update_count}")

In [10]:
# project
import pymongo

client = pymongo.MongoClient()

db = client.get_database(dbname)
collection = db.get_collection(dbname)
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter={'Date': record['Date']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

rows=307, update=0, insert=307
